In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d shaunthesheep/microsoft-catsvsdogs-dataset

Dataset URL: https://www.kaggle.com/datasets/shaunthesheep/microsoft-catsvsdogs-dataset
License(s): other
100% 785M/788M [00:06<00:00, 143MB/s]
100% 788M/788M [00:06<00:00, 121MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/microsoft-catsvsdogs-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from imutils import paths
import numpy as np
import imutils
import cv2
import os
import pickle as cPickle
from sklearn.svm import SVC
from google.colab.patches import cv2_imshow


In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from skimage.feature import graycomatrix, graycoprops
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
# Function to extract color histogram
def extract_color_histogram(image):
hist = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256]) hist = cv2.normalize(hist, hist).flatten()
return hist
# Other Features
# def extract_glcm_features(image):
# # Convert the image to grayscale
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# # Compute GLCM
# glcm = graycomatrix(gray, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
# # Extract properties
# contrast = graycoprops(glcm, 'contrast')[0, 0]
# homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
# energy = graycoprops(glcm, 'energy')[0, 0]
# return [contrast, homogeneity, energy]
# Load your data
data = []
labels = []
imagePaths = list(paths.list_images('/content/dataset_train/train')) # Replace with actual image paths
for (i, imagePath) in enumerate(imagePaths):
image = cv2.imread(imagePath)
label = imagePath.split(os.path.sep)[-1].split(".")[0]
hist = extract_color_histogram(image)
data.append(hist)
labels.append(label)
if i % 100 == 0:
print("{}/{}".format(i, len(imagePaths)))
# Convert lists to numpy arrays
data = np.array(data)
labels = np.array(labels)


In [ ]:
data.shape

In [ ]:
# Standardize the data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)


In [ ]:
# Apply PCA to reduce to 2 dimensions for visualization
pca = PCA(n_components=2)
data_pca = pca.fit_transform(data_scaled)
# Plotting the PCA results
plt.figure(figsize=(8, 6))
for label in np.unique(labels):
    indices = np.where(labels == label)
    plt.scatter(data_pca[indices, 0], data_pca[indices, 1], label=label, alpha=0.5)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('2D PCA of Image Histogram Data')
plt.legend()
plt.show()


In [ ]:
pca = PCA()
data_pca = pca.fit_transform(data_scaled)
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)
# Plotting the explained variance ratio
plt.figure(figsize=(8, 6))
plt.bar(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, alpha=0.5, align='center', label='Individual explained variance')
plt.step(range(1, len(cumulative_variance_ratio) + 1), cumulative_variance_ratio, where='mid', label='Cumulative explained variance')
plt.xlabel('Principal component index')
plt.ylabel('Explained variance ratio')
plt.title('Explained Variance Ratio by Principal Component')
plt.legend(loc='best')
plt.show()
# Determine the number of components to retain based on a threshold (e.g., 95% of variance) threshold = 0.95
num_components = np.argmax(cumulative_variance_ratio >= threshold) + 1
print(f"Number of components to retain 95% variance: {num_components}")


In [ ]:
# Standardize the data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)
# Apply PCA to reduce to 2 dimensions for visualization
pca = PCA(n_components=354)
data_pca = pca.fit_transform(data_scaled)
data_pca.shape



In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data_pca, labels, test_size=0.20, random_state=3)


In [ ]:
# Choose the SVM kernel based on the PCA plot
# Here, we'll start with linear, but you might choose 'rbf' based on the visualization svclassifier = SVC(kernel='rbf')
svclassifier.fit(X_train, y_train)


In [ ]:
predictions = svclassifier.predict(X_test)
print(classification_report(y_test, predictions))
 precision recall f1-score support
 cat 0.64 0.75 0.69 2562
 dog 0.68 0.55 0.61 2438
 accuracy 0.65 5000
 macro avg 0.66 0.65 0.65 5000
weighted avg 0.66 0.65 0.65 5000


In [ ]:
f = open("/content/model.cPickle", "wb")
f.write(cPickle.dumps(svclassifier))
f.close()


In [ ]:
model = cPickle.loads(open('/content/model.cPickle', "rb").read())

In [ ]:
# Choose the SVM kernel based on the PCA plot
# Here, we'll start with linear, but you might choose 'rbf' based on the visualization svclassifier = SVC(kernel= 'linear')
svclassifier.fit(X_train, y_train)
predictions = svclassifier.predict(X_test)
print(classification_report(y_test, predictions))
 precision recall f1-score support
 cat 0.64 0.75 0.69 2562
 dog 0.68 0.55 0.61 2438
 accuracy 0.65 5000
 macro avg 0.66 0.65 0.65 5000
weighted avg 0.66 0.65 0.65 5000


In [ ]:
test_imagePaths = list(paths.list_images('/content/dataset_test/test1'))
def predict_single_img(img_path, model, scaler, pca):
# Read the image
singleImage = cv2.imread(img_path)
print(singleImage.shape)
# Extract color histogram
hist = extract_color_histogram(singleImage).reshape(1, -1)
print(hist.shape)
# Display the image
cv2_imshow(singleImage)
# Scale the features
hist_scaled = scaler.fit_transform(hist)
# Apply PCA to reduce dimensionality to 354 features
hist_pca = pca.transform(hist_scaled)
# Predict using the model
prediction = model.predict(hist_pca)
print(prediction)
predict_single_img(test_imagePaths[0], svclassifier, scaler, pca)


In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from skimage.feature import graycomatrix, graycoprops
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
# Function to extract color histogram (you should define this function as per your feature extraction method) def extract_color_histogram(image):
hist = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256]) hist = cv2.normalize(hist, hist).flatten()
return hist
# Other Features
# def extract_glcm_features(image):
# # Convert the image to grayscale
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# # Compute GLCM
# glcm = graycomatrix(gray, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
# # Extract properties
# contrast = graycoprops(glcm, 'contrast')[0, 0]
# homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
# energy = graycoprops(glcm, 'energy')[0, 0]
# return [contrast, homogeneity, energy]
# Load your data
data = []


In [ ]:
labels = []
imagePaths = list(paths.list_images('/content/dataset_train/train')) # Replace with actual image paths for(i, imagePath) in enumerate(imagePaths):
image = cv2.imread(imagePath)
label = imagePath.split(os.path.sep)[-1].split(".")[0]
hist = extract_color_histogram(image)
data.append(hist)
if label=="cat":
labels.append(0) #cat as 0
else:
labels.append(1) #dog as 1
if i%100 == 0:
print("{}/{}".format(i, len(imagePaths)))
# Convert lists to numpy arrays
data = np.array(data)
labels = np.array(labels)


In [ ]:
# Standardize the data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)
# Apply PCA to reduce to 2 dimensions for visualization
pca = PCA(n_components=354)
data_pca = pca.fit_transform(data_scaled)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.array(data_pca), labels, test_size = 0.20, random_state=3)

In [ ]:
3 Hidden Layers (1024, 256, 128 neurons) , 1 Output Layer with 1 neuron.
import tensorflow as tf
import matplotlib.pyplot as plt
# Set random seed
tf.random.set_seed(42)
# Define the model
model = tf.keras.Sequential([
tf.keras.layers.Dense(1024, activation='relu', input_shape=(X_train.shape[1],)),
tf.keras.layers.Dense(256, activation='relu'),
tf.keras.layers.Dense(128, activation='relu'),
tf.keras.layers.Dense(1, activation='sigmoid')
])
# Compile the model
model.compile(
loss=tf.keras.losses.binary_crossentropy,
optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
metrics=['accuracy']
)
# Display the model summary to show total number of parameters
model.summary()


In [ ]:
# Train the model and store the history
history = model.fit(X_train, y_train, epochs=25, verbose=1)


In [ ]:
# Plot loss and accuracy
def plot_history(history):
plt.figure(figsize=(12, 4))
# Plot loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend()
# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy over Epochs')
plt.legend()
plt.show()
plot_history(history)

In [ ]:
print(classification_report(y_test, predictions))
 precision recall f1-score support
 0 0.62 0.85 0.71 2562
 1 0.74 0.45 0.56 2438
 accuracy 0.65 5000
 macro avg 0.68 0.65 0.64 5000
weighted avg 0.68 0.65 0.64 5000


In [ ]:
def predict_single_img(img_path, model, scaler, pca):
# Read the image
singleImage = cv2.imread(img_path)
print(singleImage.shape)
# Extract color histogram
hist = extract_color_histogram(singleImage).reshape(1, -1)
print(hist.shape)
# Display the image
cv2_imshow(singleImage)
# Scale the features
hist_scaled = scaler.fit_transform(hist)
# Apply PCA to reduce dimensionality to 354 features
hist_pca = pca.transform(hist_scaled)
# Predict using the model
prediction = model.predict(hist_pca)
if prediction>=0.5:
print("dog")
else:
print("cat")
predict_single_img(test_imagePaths[1000], model, scaler, pca)
(358, 500, 3)
(1, 512)
